In [6]:
import os
import cv2 as cv
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf
import pandas as pd
import torch.nn.functional as F
from tensorflow.keras import backend as K
!pip install pretrainedmodels
!pip install efficientnet_pytorch

In [2]:
from google.colab import drive
drive_dir = drive.mount('/content/drive')

Mounted at /content/drive


In [64]:
data_dir = '/content/drive/My Drive/mp'
weight_dir = '/content/drive/My Drive/clam_weights/pre-trained_weights'
model_dir = '/content/drive/My Drive/MP-Net/src/segmentation_models/segmentation_models_pytorch'
root_dir = "/content/drive/My Drive/MP-Net/src/segmentation_models"
images_dir = os.path.join(data_dir, 'images')
masks_dir = os.path.join(data_dir, 'masks')
unet_dir = os.path.join(model_dir, 'unet')
resized_images_dir = os.path.join(data_dir, 'resized_images')
resized_masks_dir = os.path.join(data_dir, 'resized_masks')

In [65]:
import sys
from importlib import import_module

if root_dir not in sys.path:
    sys.path.append(root_dir)

# 3. Check existence
if not os.path.isdir(unet_dir):
    print(f"Error: The 'unet' directory was not found at '{unet_dir}'.")
    sys.exit(1)

from segmentation_models_pytorch.unet.model import Unet

# Dynamically import the UNet class from unet.model
try:
    unet_model = import_module('segmentation_models_pytorch.unet.model')
    UNet = getattr(unet_model, 'Unet')  # or 'UNet' if spelled that way
except ModuleNotFoundError as e:
    print(f"Error importing the Unet model: {e}")
    sys.exit(1)
except AttributeError:
    print("Error: The 'Unet' class was not found in 'model.py'.")
    sys.exit(1)

model = UNet(encoder_name="resnet101",     # <-- Must match your checkpoint’s encoder
    encoder_weights=None,         # or "imagenet" if that was used
    in_channels=3,                # checkpoint also shows 3-channel input (conv1 has shape [64,3,7,7])
    classes=1)                     # checkpoint’s segmentation head has out_channels=1

#model_dict = model.load_state_dict(torch.load(os.path.join(weight_dir,'unet4.pth'), map_location='cpu'), strict=False)

#checkpoint = torch.load(os.path.join(weight_dir, 'unet4.pth'), map_location='cpu')
#print(checkpoint.keys())  # see what’s inside

model.load_state_dict(
    torch.load(os.path.join(weight_dir, 'unet4.pth'), map_location='cpu'),
    )

# Set to evaluation mode (if using for inference)
model.eval()

<ipython-input-65-dbb2f5fdd7f8>:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(weight_dir, 'unet4.pth'), map_location='cpu'),


Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

# Resizing and saving to Drive

In [60]:
for dir in [images_dir, masks_dir]:
  for i, img_name in enumerate(os.listdir(dir)):
    image_path = os.path.join(dir, img_name)
    img = cv.imread(image_path)
    resized_img = cv.resize(img, (256, 256), interpolation=cv.INTER_AREA)

    cv.imwrite(os.path.join(resized_images_dir if "images" in dir else resized_masks_dir, img_name), resized_img)

# Utilities

In [61]:
def show_image(img, cmap='gray'):
  plt.imshow(img, cmap=cmap)
  plt.show()

def show_all_images(*images):
  fig = plt.figure(figsize=(10, 7))
  # setting values to rows and column variables
  rows = 1
  columns = len(images)
  for i, image in enumerate(images):
    # Adds a subplot at the 1st position
    fig.add_subplot(rows, columns, i + 1)

    # showing image
    plt.imshow(image, cmap='gray')
    plt.axis('off')

def show_images(labels, *images):
  fig = plt.figure(figsize=(10, 7))
  # setting values to rows and column variables
  rows = 1
  columns = len(images)

  for i, image in enumerate(images):
    # Adds a subplot at the 1st position
    fig.add_subplot(rows, columns, i + 1)

    # showing image
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.title(labels[i] if i + 1 <= len(labels) else i)

In [62]:
from PIL import Image
def save_numpy_as_image(array, path):
    """
    Saves a numpy array as an image.

    Parameters:
        array (numpy.ndarray): The image data as a numpy array.
        path (str): The path where the image will be saved.
    """
    # If the image is binary (i.e., has a shape like (256, 256, 1)),
    # we should first remove the singleton dimension before saving it
    if array.ndim == 3 and array.shape[-1] == 1:
        array = array.squeeze(-1)

    # Convert the numpy array to a PIL image
    image = Image.fromarray(np.uint8(array * 255))

    # Save the image
    image.save(path)

    print(f"Image saved as {path}")

# Loading

In [66]:
def load_images_and_masks(images_dir, masks_dir):
  images_arr = []
  masks_arr = []
  for i, img_name in enumerate(os.listdir(masks_dir)):
    image_path = os.path.join(images_dir, img_name)
    mask_path = image_path.replace(images_dir, masks_dir)
    img = cv.imread(image_path)

    if (os.path.isfile(mask_path)):
      mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
      mask = np.expand_dims(mask, axis=-1)
      images_arr.append(img)
      masks_arr.append(mask)

  return (images_arr, masks_arr)

(raw_images, raw_masks) = load_images_and_masks(resized_images_dir, resized_masks_dir)

# Testing

In [42]:
# Example: Input tensor (ensure it matches your model's expected shape)
input_tensor = torch.randn(1, 3, 256, 256)  # Adjust dimensions as required

# Get predictions
with torch.no_grad():  # Disable gradient computation for inference
    output = model(input_tensor)

print(output)

tensor([[[[-15.0962, -19.8640, -17.2003,  ..., -17.6149, -15.4442, -10.1172],
          [-21.4769, -32.0092, -28.6532,  ..., -25.7337, -23.0573, -14.5904],
          [-19.7072, -30.8965, -31.8538,  ..., -24.2221, -23.1560, -16.3116],
          ...,
          [-16.7169, -24.4656, -23.3895,  ..., -25.9158, -25.3522, -15.9305],
          [-16.3474, -23.0582, -23.3496,  ..., -26.2157, -24.5964, -15.4051],
          [ -9.6156, -13.2047, -14.0943,  ..., -17.3262, -16.3038, -10.0858]]]])
